In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data as Data
from torch.autograd import Variable
from sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler() # define min max scaler

In [2]:
# read file
stock_path = "C:/Users/USER/Desktop/Lab/lab_minmax.csv"
stock_df = pd.read_csv(stock_path)
stock_df=stock_df.fillna(0)

In [3]:
train_set = stock_df[:2200]
test_set = stock_df[2200:]
print(train_set.shape)
print(test_set.shape)

(2200, 12)
(238, 12)


In [4]:
# every features in range(0 , 1)
train_set.describe()

,K value,D value,MACD,William,SMA,WMA,Momentum,RSI,A/D 0,CCI,Close r/f
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,0.478459,0.494744,0.515782,0.515950,0.425581,0.426475,0.464374,0.492458,0.445859,0.463808,1.000000
std,0.242761,0.231248,0.128572,0.292113,0.297391,0.296803,0.101128,0.251304,0.278231,0.159410,0.964113
min,0.004943,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.272646,0.306148,0.453617,0.258398,0.180373,0.181665,0.419330,0.292431,0.208334,0.351748,0.000000
50%,0.469031,0.482729,0.512733,0.520003,0.364709,0.364416,0.461561,0.500000,0.415185,0.467025,1.000000
75%,0.691137,0.682273,0.585561,0.777773,0.712705,0.712135,0.514771,0.687486,0.687456,0.572115,2.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000


In [5]:
# print the number of rise , flat , fall data
print("Train set Rise: %.4f"%(len(train_set[train_set["Close r/f"] == 2])*100 / len(train_set)))
print("Train set Flat: %.4f"%(len(train_set[train_set["Close r/f"] == 1])*100 / len(train_set)))
print("Train set Fall: %.4f"%(len(train_set[train_set["Close r/f"] == 0])*100 / len(train_set)))
print("Test set Rise: %.4f"%(len(test_set[test_set["Close r/f"] == 2])*100 / len(test_set)))
print("Test set Flat: %.4f"%(len(test_set[test_set["Close r/f"] == 1])*100 / len(test_set)))
print("Test set Fall: %.4f"%(len(test_set[test_set["Close r/f"] == 0])*100 / len(test_set)))


Train set Rise: 46.4545
Train set Flat: 7.0909
Train set Fall: 46.4545
Test set Rise: 49.5798
Test set Flat: 2.5210
Test set Fall: 47.8992


In [6]:
# declear training features data
features = []
for i in range(30,len(train_set)):
    x = train_set[i-30:i][["K value ","D value","William","MACD","SMA","WMA","Momentum","RSI","A/D 0","CCI"]].values
    features.append(x.tolist())
features = torch.FloatTensor(features)
features.size()

torch.Size([2170, 30, 10])

In [7]:
# declear trainging labels data
labels = []
for i in range(31,len(train_set)+1):
    x = train_set[i-1:i]["Close r/f"]
    labels.append(x.tolist())
labels = torch.LongTensor(labels).view(-1)
labels.size()

torch.Size([2170])

In [8]:
#hy parameter
torch.manual_seed(1)
EPOCH = 500
BATCH_SIZE = 64
TIME_STEP = 30
INPUT_SIZE = 10
LR = 0.001

In [9]:

#Mini-Batch

torch_dataset = Data.TensorDataset(features,labels)
train_loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    num_workers = 2,
    shuffle=True,
)

In [10]:
class autoencoder(torch.nn.Module):
    def __init__(self):
        super(autoencoder, self).__init__()
        self.encoder = torch.nn.Sequential(
            torch.nn.Linear(TIME_STEP * INPUT_SIZE, 64),
            torch.nn.ReLU(True),
            torch.nn.Linear(64, 12), 
            torch.nn.ReLU(True), 
            torch.nn.Linear(12, 3))
        self.decoder = torch.nn.Sequential(
            torch.nn.Linear(3, 12),
            torch.nn.ReLU(True),
            torch.nn.Linear(12, 64),
            torch.nn.ReLU(True), 
            torch.nn.Linear(64, TIME_STEP * INPUT_SIZE), 
            torch.nn.Tanh())

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [11]:
model = autoencoder()
print(model)

autoencoder(
  (encoder): Sequential(
    (0): Linear(in_features=300, out_features=64, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=64, out_features=12, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=12, out_features=3, bias=True)
  )
  (decoder): Sequential(
    (0): Linear(in_features=3, out_features=12, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=12, out_features=64, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=64, out_features=300, bias=True)
    (5): Tanh()
  )
)


In [12]:
# define optimizer and loss function 
optimizer = torch.optim.Adam(model.parameters(), lr=LR,weight_decay=0.0005)
# adject learning rate . when loss don't fall , lr = lr * factor  , min lr = 0.0001
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,factor = 0.9,min_lr=0.0001)
# crossentroy loss 
loss_func = torch.nn.MSELoss()

In [13]:

#delcear testing features and labels

test_features = []
for i in range(30,len(test_set)):
    x = test_set[i-30:i][["K value ","D value","William","MACD","SMA","WMA","Momentum","RSI","A/D 0","CCI"]].values
    test_features.append(x.tolist())
test_features = torch.FloatTensor(test_features)
test_labels = []
for i in range(31,len(test_set)+1):
    x = test_set[i-1:i]["Close r/f"]
    test_labels.append(x.tolist())
test_labels = torch.FloatTensor(test_labels).view(-1)

In [14]:
'''
Training 
'''
LOSS = []
ACC = []
TRAIN_ACC = []
for epoch in range(EPOCH):
    loss_total = 0
    for step,(inputs,targets) in enumerate(train_loader):
        inputs = inputs.view(-1,TIME_STEP * INPUT_SIZE)
        #reshape the features to (batch,time_step,input_size)
        inputs_n = inputs.view(-1,TIME_STEP * INPUT_SIZE)
        # transform numpy array to torch tensor
        inputs_n = inputs_n.numpy()
        noise = np.random.randint(0,1,TIME_STEP * INPUT_SIZE)
        inputs_n = inputs_n + noise
        inputs_n = torch.Tensor(inputs_n)
        # start trainnig 
        output = model(inputs_n)
        # calculate loss  (cross entroy)
        loss = loss_func(output,inputs)
        # clear the gradients of all optimized variables(from last training)
        optimizer.zero_grad()
        
        # back propagation
        loss.backward()
        optimizer.step()
        # sum of loss
        loss_total = loss_total + loss
        
        # print training info every 10 steps
        if((step+1) %10 == 0):
            # average of loss in 10 steps
            avg = loss_total / 10
            LOSS.append(avg.tolist())
            

            # print the epoch , steps , average loss , accuracy 
            print("Epoch: %4d|steps: %4d|Train Avg Loss: %.4f |lr = %.5f"
                  %(epoch+1,step+1,avg,optimizer.param_groups[0]['lr']))
            
            # inital variable
            loss_total = 0
    # updata learning rate
    scheduler.step(loss)
        


Epoch:    1|steps:   10|Train Avg Loss: 0.2609 |lr = 0.00100
Epoch:    1|steps:   20|Train Avg Loss: 0.2022 |lr = 0.00100
Epoch:    1|steps:   30|Train Avg Loss: 0.1316 |lr = 0.00100
Epoch:    2|steps:   10|Train Avg Loss: 0.0696 |lr = 0.00100
Epoch:    2|steps:   20|Train Avg Loss: 0.0575 |lr = 0.00100
Epoch:    2|steps:   30|Train Avg Loss: 0.0545 |lr = 0.00100
Epoch:    3|steps:   10|Train Avg Loss: 0.0541 |lr = 0.00100
Epoch:    3|steps:   20|Train Avg Loss: 0.0530 |lr = 0.00100
Epoch:    3|steps:   30|Train Avg Loss: 0.0522 |lr = 0.00100
Epoch:    4|steps:   10|Train Avg Loss: 0.0520 |lr = 0.00100
Epoch:    4|steps:   20|Train Avg Loss: 0.0504 |lr = 0.00100
Epoch:    4|steps:   30|Train Avg Loss: 0.0504 |lr = 0.00100
Epoch:    5|steps:   10|Train Avg Loss: 0.0481 |lr = 0.00100
Epoch:    5|steps:   20|Train Avg Loss: 0.0466 |lr = 0.00100
Epoch:    5|steps:   30|Train Avg Loss: 0.0469 |lr = 0.00100
Epoch:    6|steps:   10|Train Avg Loss: 0.0439 |lr = 0.00100
Epoch:    6|steps:   20|

Epoch:   46|steps:   10|Train Avg Loss: 0.0315 |lr = 0.00081
Epoch:   46|steps:   20|Train Avg Loss: 0.0324 |lr = 0.00081
Epoch:   46|steps:   30|Train Avg Loss: 0.0324 |lr = 0.00081
Epoch:   47|steps:   10|Train Avg Loss: 0.0315 |lr = 0.00081
Epoch:   47|steps:   20|Train Avg Loss: 0.0317 |lr = 0.00081
Epoch:   47|steps:   30|Train Avg Loss: 0.0324 |lr = 0.00081
Epoch:   48|steps:   10|Train Avg Loss: 0.0322 |lr = 0.00081
Epoch:   48|steps:   20|Train Avg Loss: 0.0322 |lr = 0.00081
Epoch:   48|steps:   30|Train Avg Loss: 0.0318 |lr = 0.00081
Epoch:   49|steps:   10|Train Avg Loss: 0.0318 |lr = 0.00081
Epoch:   49|steps:   20|Train Avg Loss: 0.0319 |lr = 0.00081
Epoch:   49|steps:   30|Train Avg Loss: 0.0326 |lr = 0.00081
Epoch:   50|steps:   10|Train Avg Loss: 0.0319 |lr = 0.00073
Epoch:   50|steps:   20|Train Avg Loss: 0.0320 |lr = 0.00073
Epoch:   50|steps:   30|Train Avg Loss: 0.0317 |lr = 0.00073
Epoch:   51|steps:   10|Train Avg Loss: 0.0320 |lr = 0.00073
Epoch:   51|steps:   20|

Epoch:   91|steps:   10|Train Avg Loss: 0.0310 |lr = 0.00053
Epoch:   91|steps:   20|Train Avg Loss: 0.0309 |lr = 0.00053
Epoch:   91|steps:   30|Train Avg Loss: 0.0306 |lr = 0.00053
Epoch:   92|steps:   10|Train Avg Loss: 0.0309 |lr = 0.00053
Epoch:   92|steps:   20|Train Avg Loss: 0.0305 |lr = 0.00053
Epoch:   92|steps:   30|Train Avg Loss: 0.0306 |lr = 0.00053
Epoch:   93|steps:   10|Train Avg Loss: 0.0310 |lr = 0.00053
Epoch:   93|steps:   20|Train Avg Loss: 0.0309 |lr = 0.00053
Epoch:   93|steps:   30|Train Avg Loss: 0.0306 |lr = 0.00053
Epoch:   94|steps:   10|Train Avg Loss: 0.0313 |lr = 0.00053
Epoch:   94|steps:   20|Train Avg Loss: 0.0307 |lr = 0.00053
Epoch:   94|steps:   30|Train Avg Loss: 0.0300 |lr = 0.00053
Epoch:   95|steps:   10|Train Avg Loss: 0.0308 |lr = 0.00053
Epoch:   95|steps:   20|Train Avg Loss: 0.0305 |lr = 0.00053
Epoch:   95|steps:   30|Train Avg Loss: 0.0308 |lr = 0.00053
Epoch:   96|steps:   10|Train Avg Loss: 0.0307 |lr = 0.00048
Epoch:   96|steps:   20|

Epoch:  136|steps:   10|Train Avg Loss: 0.0291 |lr = 0.00035
Epoch:  136|steps:   20|Train Avg Loss: 0.0291 |lr = 0.00035
Epoch:  136|steps:   30|Train Avg Loss: 0.0291 |lr = 0.00035
Epoch:  137|steps:   10|Train Avg Loss: 0.0289 |lr = 0.00035
Epoch:  137|steps:   20|Train Avg Loss: 0.0295 |lr = 0.00035
Epoch:  137|steps:   30|Train Avg Loss: 0.0292 |lr = 0.00035
Epoch:  138|steps:   10|Train Avg Loss: 0.0292 |lr = 0.00035
Epoch:  138|steps:   20|Train Avg Loss: 0.0292 |lr = 0.00035
Epoch:  138|steps:   30|Train Avg Loss: 0.0291 |lr = 0.00035
Epoch:  139|steps:   10|Train Avg Loss: 0.0289 |lr = 0.00035
Epoch:  139|steps:   20|Train Avg Loss: 0.0293 |lr = 0.00035
Epoch:  139|steps:   30|Train Avg Loss: 0.0294 |lr = 0.00035
Epoch:  140|steps:   10|Train Avg Loss: 0.0294 |lr = 0.00035
Epoch:  140|steps:   20|Train Avg Loss: 0.0290 |lr = 0.00035
Epoch:  140|steps:   30|Train Avg Loss: 0.0291 |lr = 0.00035
Epoch:  141|steps:   10|Train Avg Loss: 0.0297 |lr = 0.00035
Epoch:  141|steps:   20|

Epoch:  181|steps:   10|Train Avg Loss: 0.0290 |lr = 0.00025
Epoch:  181|steps:   20|Train Avg Loss: 0.0285 |lr = 0.00025
Epoch:  181|steps:   30|Train Avg Loss: 0.0285 |lr = 0.00025
Epoch:  182|steps:   10|Train Avg Loss: 0.0289 |lr = 0.00025
Epoch:  182|steps:   20|Train Avg Loss: 0.0289 |lr = 0.00025
Epoch:  182|steps:   30|Train Avg Loss: 0.0284 |lr = 0.00025
Epoch:  183|steps:   10|Train Avg Loss: 0.0289 |lr = 0.00025
Epoch:  183|steps:   20|Train Avg Loss: 0.0285 |lr = 0.00025
Epoch:  183|steps:   30|Train Avg Loss: 0.0288 |lr = 0.00025
Epoch:  184|steps:   10|Train Avg Loss: 0.0282 |lr = 0.00025
Epoch:  184|steps:   20|Train Avg Loss: 0.0289 |lr = 0.00025
Epoch:  184|steps:   30|Train Avg Loss: 0.0291 |lr = 0.00025
Epoch:  185|steps:   10|Train Avg Loss: 0.0291 |lr = 0.00025
Epoch:  185|steps:   20|Train Avg Loss: 0.0284 |lr = 0.00025
Epoch:  185|steps:   30|Train Avg Loss: 0.0286 |lr = 0.00025
Epoch:  186|steps:   10|Train Avg Loss: 0.0291 |lr = 0.00025
Epoch:  186|steps:   20|

...

Epoch:  316|steps:   10|Train Avg Loss: 0.0280 |lr = 0.00010
Epoch:  316|steps:   20|Train Avg Loss: 0.0275 |lr = 0.00010
Epoch:  316|steps:   30|Train Avg Loss: 0.0275 |lr = 0.00010
Epoch:  317|steps:   10|Train Avg Loss: 0.0271 |lr = 0.00010
Epoch:  317|steps:   20|Train Avg Loss: 0.0278 |lr = 0.00010
Epoch:  317|steps:   30|Train Avg Loss: 0.0278 |lr = 0.00010
Epoch:  318|steps:   10|Train Avg Loss: 0.0277 |lr = 0.00010
Epoch:  318|steps:   20|Train Avg Loss: 0.0278 |lr = 0.00010
Epoch:  318|steps:   30|Train Avg Loss: 0.0275 |lr = 0.00010
Epoch:  319|steps:   10|Train Avg Loss: 0.0276 |lr = 0.00010
Epoch:  319|steps:   20|Train Avg Loss: 0.0277 |lr = 0.00010
Epoch:  319|steps:   30|Train Avg Loss: 0.0276 |lr = 0.00010
Epoch:  320|steps:   10|Train Avg Loss: 0.0276 |lr = 0.00010
Epoch:  320|steps:   20|Train Avg Loss: 0.0270 |lr = 0.00010
Epoch:  320|steps:   30|Train Avg Loss: 0.0280 |lr = 0.00010
Epoch:  321|steps:   10|Train Avg Loss: 0.0279 |lr = 0.00010
Epoch:  321|steps:   20|

Epoch:  361|steps:   10|Train Avg Loss: 0.0268 |lr = 0.00010
Epoch:  361|steps:   20|Train Avg Loss: 0.0264 |lr = 0.00010
Epoch:  361|steps:   30|Train Avg Loss: 0.0267 |lr = 0.00010
Epoch:  362|steps:   10|Train Avg Loss: 0.0262 |lr = 0.00010
Epoch:  362|steps:   20|Train Avg Loss: 0.0267 |lr = 0.00010
Epoch:  362|steps:   30|Train Avg Loss: 0.0269 |lr = 0.00010
Epoch:  363|steps:   10|Train Avg Loss: 0.0270 |lr = 0.00010
Epoch:  363|steps:   20|Train Avg Loss: 0.0267 |lr = 0.00010
Epoch:  363|steps:   30|Train Avg Loss: 0.0265 |lr = 0.00010
Epoch:  364|steps:   10|Train Avg Loss: 0.0269 |lr = 0.00010
Epoch:  364|steps:   20|Train Avg Loss: 0.0264 |lr = 0.00010
Epoch:  364|steps:   30|Train Avg Loss: 0.0265 |lr = 0.00010
Epoch:  365|steps:   10|Train Avg Loss: 0.0263 |lr = 0.00010
Epoch:  365|steps:   20|Train Avg Loss: 0.0267 |lr = 0.00010
Epoch:  365|steps:   30|Train Avg Loss: 0.0266 |lr = 0.00010
Epoch:  366|steps:   10|Train Avg Loss: 0.0266 |lr = 0.00010
Epoch:  366|steps:   20|

Epoch:  406|steps:   10|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  406|steps:   20|Train Avg Loss: 0.0265 |lr = 0.00010
Epoch:  406|steps:   30|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  407|steps:   10|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  407|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  407|steps:   30|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  408|steps:   10|Train Avg Loss: 0.0265 |lr = 0.00010
Epoch:  408|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  408|steps:   30|Train Avg Loss: 0.0258 |lr = 0.00010
Epoch:  409|steps:   10|Train Avg Loss: 0.0257 |lr = 0.00010
Epoch:  409|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  409|steps:   30|Train Avg Loss: 0.0264 |lr = 0.00010
Epoch:  410|steps:   10|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  410|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  410|steps:   30|Train Avg Loss: 0.0265 |lr = 0.00010
Epoch:  411|steps:   10|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  411|steps:   20|

Epoch:  451|steps:   10|Train Avg Loss: 0.0258 |lr = 0.00010
Epoch:  451|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  451|steps:   30|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  452|steps:   10|Train Avg Loss: 0.0263 |lr = 0.00010
Epoch:  452|steps:   20|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  452|steps:   30|Train Avg Loss: 0.0258 |lr = 0.00010
Epoch:  453|steps:   10|Train Avg Loss: 0.0257 |lr = 0.00010
Epoch:  453|steps:   20|Train Avg Loss: 0.0262 |lr = 0.00010
Epoch:  453|steps:   30|Train Avg Loss: 0.0262 |lr = 0.00010
Epoch:  454|steps:   10|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  454|steps:   20|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  454|steps:   30|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  455|steps:   10|Train Avg Loss: 0.0256 |lr = 0.00010
Epoch:  455|steps:   20|Train Avg Loss: 0.0265 |lr = 0.00010
Epoch:  455|steps:   30|Train Avg Loss: 0.0263 |lr = 0.00010
Epoch:  456|steps:   10|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  456|steps:   20|

Epoch:  496|steps:   10|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  496|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  496|steps:   30|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  497|steps:   10|Train Avg Loss: 0.0258 |lr = 0.00010
Epoch:  497|steps:   20|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  497|steps:   30|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  498|steps:   10|Train Avg Loss: 0.0258 |lr = 0.00010
Epoch:  498|steps:   20|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  498|steps:   30|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  499|steps:   10|Train Avg Loss: 0.0260 |lr = 0.00010
Epoch:  499|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  499|steps:   30|Train Avg Loss: 0.0258 |lr = 0.00010
Epoch:  500|steps:   10|Train Avg Loss: 0.0259 |lr = 0.00010
Epoch:  500|steps:   20|Train Avg Loss: 0.0261 |lr = 0.00010
Epoch:  500|steps:   30|Train Avg Loss: 0.0258 |lr = 0.00010
